# 自动微分

假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度

In [2]:
"""
梯度追踪:
    作用：标记张量 x 需要计算梯度
    关键点：必须在前向传播前设置
    下划线 _：表示原地（in-place）修改
"""
x.requires_grad_(True)
"""
存储计算的梯度
    作用：反向传播后，这里存储 x 的梯度值
    初始值：None（未进行反向传播前）
"""
x.grad

现在计算$y$：$y=2\sum_{i=0}^{3}x_{i}^{2}$，$\frac{\partial y}{\partial x_{i}}=4 x_{i}$

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算`x`的另一个函数，前向：$y=\sum_{i=0}^{3}x_{i}$，$\frac{\partial y}{\partial x_{i}}=1$ 

In [6]:
# x.grad 不会自动清零，多次 backward() 会累积
# 训练循环中必须使用 optimizer.zero_grad() 或 x.grad.zero_()清零梯度
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和，$y_{i}=x_{i}^{2}$，$loss=\sum y_{i}=\sum x_{i}^{2}$，$\frac{\partial loss}{\partial x_{i}}=2x_{i}$

In [7]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [8]:
x.grad.zero_()
y = x * x
# ❗️关键：u = y 的值，但脱离计算图,效果：z=u*x中，PyTorch将u视为常量，梯度不会流经u回到 x。
u = y.detach()
z = u * x # z=x²·x(x²被视为常数C)

z.sum().backward()
x.grad == u # tensor([0., 1., 4., 9.])  导数x²

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [10]:
"""
    对于齐次函数，梯度等于输出除以输入。
    size=()  ——  0维张量（标量） () 表示空元组，代表0维
    结果是一个标量，不是一维数组,类似 Python 的 float，但保留 PyTorch 张量特性
"""
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True) # 生成正态分布随机数
d = f(a) # d=C*a
d.backward()

a.grad == d / a

tensor(True)